In [79]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
%%javascript

require(["https://unpkg.com/higlass-multi-tileset"],
    function(hglib) {

});

require(["https://unpkg.com/higlass-dynseq@0.1.4/dist/higlass-dynseq.umd.js"],
    function(hglib) {
});

require(["https://unpkg.com/higlass-transcripts/dist/higlass-transcripts.js"],
    function(hglib) {
});

require(["https://unpkg.com/higlass-arcs/dist/higlass-arcs.min.js"],
    function(hglib) {
});

<IPython.core.display.Javascript object>

In [81]:
import higlass
from higlass.client import Track, View

data = {
  "type": "multi-tileset",
  "configs": [
    {
      "server": "https://resgen.io/api/v1",
      "tilesetUid": "GsjRN353Qmm-TmEohvLNBA"
    },
    {
      "server": "https://resgen.io/api/v1",
      "tilesetUid": "Z6_3sHNKQK2exNOFJXcshg"
    }
  ]
}

t = Track(type='horizontal-dynseq', position='top', data=data, height=100)

display, server, viewconf = higlass.display(
    [View([Track('top-axis'), t], initialXDomain=[
        1474922620.0091968,
        1474922681.829122
      ])
], fuse=False)
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'd3i469W1RHGW1-2Z7Ixgyw', 'tracks': {'top': [{'ty…

In [82]:
import os

rgl = rg.connect(os.environ.get('RESGEN_USERNAME'), os.environ.get('RESGEN_PASSWORD'), host='http://localhost:8000')

In [83]:
projectl = rgl.find_or_create_project('scratch')
projectl.sync_viewconf(viewconf, 'test view')

In [84]:
def dynseq_track(sequences_track, bigwig_track, height=60, **options):
    data = {
      "type": "multi-tileset",
      "configs": [
        {
          "server": f"{bigwig_track.conn.host}/api/v1",
          "tilesetUid": bigwig_track.uuid
        },
        {
          "server": f"{sequences_track.conn.host}/api/v1",
          "tilesetUid": sequences_track.uuid
        },
      ]
    }
    return Track(type='horizontal-dynseq', position='top', data=data, height=height, options=options)

In [106]:
import resgen as rg
rgc = rg.connect()

gene_annotations_project = rgc.find_or_create_project('gene-annotations', group='pete')
chromsizes = rgc.find_datasets(
    project=gene_annotations_project, datatype='chromsizes', assembly='hg38'
  )[0]

chrominfo = rgc.get_chrominfo(chromsizes)

gene_annotations = rgc.find_datasets(
    project=rgc.find_or_create_project('gene-annotations', group='pete'),
    datatype='gene-annotations', assembly='hg38'
)[0]
sequences_track = rgc.find_datasets(
    project=rgc.find_or_create_project('assemblies', group='pete'),
    datatype='sequence', assembly='hg38'
)[0]

hg38_transcripts = rgc.find_datasets(
    "Gene Transcripts (hg38)",
    project=rgc.find_or_create_project('gene-annotations', group='pete')                                
)[0]

WjLRf-GC: Gene Transcripts (hg38)

In [110]:
gene_annotations_track = gene_annotations.hg_track(
    height=50,
    geneAnnotationHeight=8
)
transcripts_track = hg38_transcripts.hg_track(
            track_type='horizontal-transcripts', position='top',
            height=30)

In [86]:
encode_project = rgc.find_or_create_project('encode-demo', group='kundaje-lab')

In [157]:
gata1_profile = rgc.find_datasets(
    "GATA1.K562.ENCSR000EFT.profile.bw",
    project=encode_project
)[0]
gata1_counts = rgc.find_datasets(
    "GATA1.K562.ENCSR000EFT.chr11.counts.bw",
    project=encode_project
)[0]

gata2_profile = rgc.find_datasets(
    "GATA2.K562.ENCSR000BKM.profile.bw",
    project=encode_project
)[0]
gata2_counts = rgc.find_datasets(
    "GATA2.K562.ENCSR000BKM.chr11.counts.bw",
    project=encode_project
)[0]

dnase_profile = rgc.find_datasets(
    "K562.DNase.ENCSR000EOT.chr11.profile.bw",
    project=encode_project
)[0]
dnase_counts = rgc.find_datasets(
    "K562.DNase.ENCSR000EOT.chr11.count.bw",
    project=encode_project
)[0]

nfe1_profile = rgc.find_datasets(
    "NFE2.K562.ENCSR000FCC.profile.bw",
    project=encode_project
)[0]

tal1_counts = rgc.find_datasets(
    "TAL1.K562.ENCSR000EHB.chr11.counts.bw",
    project=encode_project
)[0]
tal1_profile = rgc.find_datasets(
    "TAL1.K562.ENCSR000EHB.profile.bw",
    project=encode_project
)[0]

usf1_counts = rgc.find_datasets(
    "USF1.K562.ENCSR000BKT.chr11.counts.bw",
    project=encode_project
)[0]
usf1_profile = rgc.find_datasets(
    "USF1.K562.ENCSR000BKT.profile.bw",
    project=encode_project
)[0]

In [124]:
dnase_pred = rgc.find_datasets(
        "K562.DNase.ENCSR000EOT.chr11.pred.bw",
        project=encode_project)[0].hg_track(
            name="DNase predicted",
            height=20,
            barFillColor="blue")
danse_obs = rgc.find_datasets(
        "K562.DNase.ENCSR000EOT.bw",
        project=encode_project)[0].hg_track(
            name="DNase observed",
            height=20,
            barFillColor="blue")

In [230]:
def track_options(name, strand):
    return {
        "track_type": "horizontal-line",
        "height": 35,
        "lineStrokeColor": "blue" if strand == '+' else "orange",
        "name": name,
        "nanAsZero": True
    }

def line_track(filename, name, strand):
    return rgc.find_datasets(
        filename,
        project=encode_project)[0].hg_track(**track_options(name, strand))

In [232]:
nfe2_obs_minus = line_track("NFE2.K562.ENCSR000FCC_minus.bigWig","NFE2 observed", '-')
nfe2_obs_plus = line_track("NFE2.K562.ENCSR000FCC_plus.bigWig","NFE2 observed", '+')

tal1_obs_minus = line_track("TAL1.K562.ENCSR000EHB_minus.bigWig", "TAL1 observed", '-')
tal1_obs_plus = line_track("TAL1.K562.ENCSR000EHB_plus.bigWig", "TAL1 observed", '+')

usf1_obs_minus = line_track("USF1.K562.ENCSR000BKT_minus.bigWig", "USF1 observed", '-')
usf1_obs_plus = line_track("USF1.K562.ENCSR000BKT_plus.bigWig", "USF observed", '+')

gata1_obs_minus = line_track("GATA1.K562.ENCSR000EFT_minus.bigWig", "GATA1 observed", '-')
gata1_obs_plus = line_track("GATA1.K562.ENCSR000EFT_plus.bigWig", "GATA1 observed", '+')

gata2_obs_minus = line_track("GATA2.K562.ENCSR000BKM_minus.bigWig", "GATA2 observed", '-')
gata2_obs_plus = line_track("GATA2.K562.ENCSR000BKM_plus.bigWig", "GATA2 observed", '+')

In [233]:
nfe2_pred_minus = line_track("NFE2.K562.ENCSR000FCC.minus.bw", "NFE2 predicted", '-')
nfe2_pred_plus = line_track("NFE2.K562.ENCSR000FCC.plus.bw","NFE2 predicted", '+')

tal1_pred_minus = line_track("TAL1.K562.ENCSR000EHB.minus.bw", "TAL1 predicted", '-')
tal1_pred_plus = line_track("TAL1.K562.ENCSR000EHB.plus.bw", "TAL1 predicted", '+')

usf1_pred_minus = line_track("USF1.K562.ENCSR000BKT.minus.bw", "USF1 predicted", '-')
usf1_pred_plus = line_track("USF1.K562.ENCSR000BKT.plus.bw", "USF1 predicted", '+')

gata1_pred_minus = line_track("GATA1.K562.ENCSR000EFT.minus.bw", "GATA1 predicted", '-')
gata1_pred_plus = line_track("GATA1.K562.ENCSR000EFT.plus.bw", "GATA1 predicted", '+')

gata2_pred_minus = line_track("GATA2.K562.ENCSR000BKM.minus.bw", "GATA2 predicted", '-')
gata2_pred_plus = line_track("GATA2.K562.ENCSR000BKM.plus.bw", "GATA2 predicted", '+')

In [234]:
dynseq_height = 50

display, server, viewconf = higlass.display(
    [View([
        transcripts_track,
        chromsizes.hg_track(height=30),
        gene_annotations_track,
        danse_obs,
        dnase_pred,
        nfe2_obs_plus + nfe2_obs_minus,
        nfe2_pred_plus + nfe2_pred_minus,
        gata1_obs_plus + gata1_obs_minus,
        gata1_pred_plus + gata1_pred_minus,
        dynseq_track(
            sequences_track, gata1_profile,
            height=dynseq_height,
            name="GATA1 profile"
        ),
        dynseq_track(
            sequences_track, gata1_counts,
            height=dynseq_height,
            name="GATA2 counts"
        ),
        gata2_obs_plus + gata2_obs_minus,
        gata2_pred_plus + gata2_pred_minus,
        dynseq_track(
            sequences_track, gata2_profile,
            height=dynseq_height,
            name="GATA2 profile"
        ),
        tal1_obs_plus + tal1_obs_minus,
        tal1_pred_plus + tal1_pred_minus,
        dynseq_track(sequences_track, tal1_profile, height=dynseq_height,
                     name="TAIL1 profile"
                    ),
        dynseq_track(sequences_track, tal1_counts, height=dynseq_height,
                    name="TAIL1 counts"),
        usf1_obs_plus + usf1_obs_minus,
        usf1_pred_plus + usf1_pred_minus,
        dynseq_track(sequences_track, usf1_profile, height=dynseq_height,
                    name="USF1 profile"),
        dynseq_track(sequences_track, usf1_counts, height=dynseq_height,
                    name="USF1 counts"),
    ], initialXDomain=[
        chrominfo.to_abs('chr11', 5280637),
        chrominfo.to_abs('chr11', 5280800)
      ])
], fuse=False)
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'FwNh9b8JRlCs3gMQHEvbew', 'tracks': {'top': [{'ty…

In [228]:
encode_project.sync_viewconf(viewconf, 'Figure 1')

In [204]:
import bbi
import numpy as np

data = bbi.fetch(
    'http://mitra.stanford.edu/kundaje/surag/dynseq/beta_globin/observed_bigWigs/GATA2.K562.ENCSR000BKM_minus.bigWig',
    'chr11',
    5280007, 5280830, missing=np.nan)



## Figure 2

In [87]:
encode_project2 = rgc.find_or_create_project('encode-demo-2', group='kundaje-lab')

In [88]:
cluster = 1
rgc.find_datasets(f'cluster_idx{cluster}.pval.signal.bw', project=encode_project2)[0]

NSbBLOJi: cluster_idx1.pval.signal.bw

In [89]:
import pandas as pd

t = pd.read_table('../cluster.tsv')

t.head()

,new_cluster,colour,description
0,1,#B03743,Fibroblast
1,2,#7C2730,Fibroblast-like
2,3,#E85F6D,Fibroblast-like
3,4,#CF175A,Fibroblast-like
4,5,#F01D35,Fibroblast-like


In [139]:
signal_tracks = []
signal_datasets = []

importance_tracks = []
importance_datasets = []

track_opts = {
    "track_type": "bar",
    "height": 30
}

for i,x in list(t.iterrows()):
    if x['new_cluster'] not in [1, 8, 11,12,13,14,15]:
        # subset the tracks as per Surag
        continue
        
    signal_dataset = rgc.find_datasets(
        f"cluster_idx{x['new_cluster']}.pval.signal.bw", project=encode_project2
    )[0]
    importance_dataset = rgc.find_datasets(
        f"cluster_idx{x['new_cluster']}.counts.importance.bw", project=encode_project2
    )[0]
    signal_tracks += [signal_dataset.hg_track(
        **track_opts, barFillColor=x['colour'],
        name=x['description'],
        valueScaleMin=0, valueScaleMax=40)]
    importance_tracks += [importance_dataset.hg_track(**track_opts, barFillColor=x['colour'], name=x['description'])]
    
    signal_datasets += [signal_dataset]
    importance_datasets += [importance_dataset]


In [140]:
bed_track = rgc.find_datasets(
    'overlap.peaks.resolved.500.truncated.bed.bgz', project=encode_project2
)[0].hg_track(track_type='bedlike', height=15)
arcs_track = rgc.find_datasets(
    'NANOG.links.bed', project=encode_project2
)[0].hg_track(track_type='1d-arcs', position='top', height=35)

In [141]:
from higlass import display

datasets = importance_datasets
tracks = importance_tracks
dynseq_options = {
    "labelPosition": "hidden"
}

chromsizes_track = chromsizes.hg_track(
    height=30,
    tickPositions="ends")

top_tracks = [gene_annotations_track, arcs_track, bed_track]
top_tracks_height = sum([t.conf['height'] for t in top_tracks])
empty_tracks = [Track(track_type='empty', position='top', height=top_tracks_height)]
print('top_tracks_height', top_tracks_height)

signal_view = View([chromsizes_track] + top_tracks + signal_tracks, initialXDomain=[
            chrominfo.to_abs('chr12', 7635689),
            chrominfo.to_abs('chr12', 7810787)
          ], x=3, width=6, height=6)

dynseq_view1 = View(
    [chromsizes_track] + empty_tracks +
    [dynseq_track(sequences_track, d, height=30, **dynseq_options) for d in datasets],
          initialXDomain=[
            chrominfo.to_abs('chr12', 7638839),
            chrominfo.to_abs('chr12', 7638877)
          ], x=0, width=3, height=6)
dynseq_view2 = View(
    [chromsizes_track] + empty_tracks +
    [dynseq_track(sequences_track, d, height=30, **dynseq_options) for d in datasets],
          initialXDomain=[
            chrominfo.to_abs('chr12', 7807035),
            chrominfo.to_abs('chr12', 7807075)
          ], x=9, width=3, height=6)

dynseq_view3 = View(
    [chromsizes_track] +
    [dynseq_track(sequences_track, d, height=30, **dynseq_options) for d in datasets],
          initialXDomain=[
            chrominfo.to_abs('chr12', 7716560),
            chrominfo.to_abs('chr12', 7716600)
          ], x=1, width=3, y=6, height=6)

dynseq_view4 = View(
    [chromsizes_track] +
    [dynseq_track(sequences_track, d, height=30, **dynseq_options) for d in datasets],
          initialXDomain=[
            chrominfo.to_abs('chr12', 7758383),
            chrominfo.to_abs('chr12', 7758497)
          ], x=4, width=5, y=6, height=6)

dynseq_view5 = View(
    [chromsizes_track] +
    [dynseq_track(sequences_track, d, height=30, **dynseq_options) for d in datasets],
          initialXDomain=[
            chrominfo.to_abs('chr12', 7789247),
            chrominfo.to_abs('chr12', 7789308)
          ], x=9, width=3, y=6, height=6)

d,s,v = display([
        signal_view, dynseq_view1, dynseq_view2,
        dynseq_view3, dynseq_view4, dynseq_view5
    ], fuse=False, )
# d,s,v = display([
#         signal_view, dynseq_view1], fuse=False, )
        
d

top_tracks_height 100


HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'Ooa4yzZRTuWVq1-zoTW5UA', 'tracks': {'top': [{'ty…

In [147]:
import json
import subprocess 

data = json.dumps(v.to_dict())

subprocess.run("pbcopy", universal_newlines=True, input=data)

CompletedProcess(args='pbcopy', returncode=0)